In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="17rFuCNZUUY1xHrMq1WTamV-JWh_IDZe8", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/04_00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 The RLAnything Closed Loop: Co-Optimizing Policy, Reward Model, and Environment

*Part 4 of the Vizuara series on OpenClaw-RL*
*Estimated time: 60 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/openclaw-rl/practice/4/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Why Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_01_why_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

In the previous notebooks, we built two training paradigms: Binary RL (scalar feedback) and OPD (token-level feedback). Both treat the **Process Reward Model (PRM)** as a fixed judge — the policy improves, but the judge stays the same.

But what if the judge itself could get better over time?

Think about it: as the policy improves, it generates higher-quality responses. These higher-quality responses contain more nuanced information for the reward model to learn from. And a better reward model provides more precise feedback for the policy.

This is the **RLAnything** framework — a closed loop where **policy**, **reward model**, and **environment** all co-optimize simultaneously.

By the end of this notebook, you will have implemented:
- **Integrated rewards** combining outcome signals with step-wise PRM scores
- **Consistency feedback** that trains the reward model alongside the policy
- **Automatic difficulty adaptation** that keeps the environment challenging
- A complete **closed-loop training pipeline** where all three components improve together
- Visualization of the co-evolution dynamics

In [ ]:
# 🎯 Teaser: Watch all three components improve together
# Step 0:   Policy: 45% success | PRM accuracy: 65% | Difficulty: 0.3
# Step 50:  Policy: 62% success | PRM accuracy: 73% | Difficulty: 0.5
# Step 100: Policy: 78% success | PRM accuracy: 82% | Difficulty: 0.7
# Step 150: Policy: 85% success | PRM accuracy: 88% | Difficulty: 0.8

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_02_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Let us use an analogy.

Imagine a **student**, a **teacher**, and a **textbook author** working together:

- **The student** (policy) solves problems from the textbook
- **The teacher** (reward model) grades the student's work
- **The textbook author** (environment) creates new problems

At first:
- The student is mediocre, solving only easy problems
- The teacher's grading is somewhat inaccurate
- The textbook has a mix of easy and hard problems

Over time, something beautiful happens:
- The student improves → now the easy problems are too easy
- The teacher sees better work → learns to distinguish good from great (not just good from bad)
- The textbook author notices the student acing easy problems → generates harder ones
- The harder problems force the student to improve further
- And the cycle continues...

This is **co-optimization**. No single component drives improvement alone — they lift each other up.

### 🤔 Think About This

What would happen if we only improved the policy but kept the reward model fixed? At some point, the policy would surpass the reward model's ability to evaluate it. The reward model would assign random scores to high-quality responses, and training would plateau or even degrade. This is why co-optimization matters.

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_03_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### 3.1 Integrated Reward

Instead of relying solely on the PRM, we combine **outcome rewards** (task-level success/failure) with **step-wise PRM scores**:

$$R_{\tau_i} = O_\tau + \frac{\lambda}{m}\sum_{j=1}^{m} S_{\tau_i, j}$$

where:
- $O_\tau \in \{-1, +1\}$ is the outcome reward (did the task succeed overall?)
- $S_{\tau_i, j} \in \{-1, +1\}$ is the $j$-th PRM evaluation for step $i$
- $m$ is the number of PRM evaluations
- $\lambda$ (default 1) balances outcome vs step-wise signals

Computationally: the integrated reward augments the binary outcome with fine-grained step scores. A step that was individually good ($S > 0$) in a failed task ($O = -1$) gets a less harsh penalty than a step that was individually bad.

Example: Task succeeded ($O = +1$), 3 PRM evaluations for this step: $[+1, +1, -1]$.

$R_{\tau_i} = 1 + \frac{1}{3}(1 + 1 + (-1)) = 1 + 0.33 = 1.33$

### 3.2 Consistency Feedback for the Reward Model

How does the PRM improve? Through a **self-consistency signal**:

$$R_{S_{\tau_i, j}} = R_{\tau_i} \cdot S_{\tau_i, j}$$

Computationally: if the PRM agrees with the integrated reward (both positive or both negative), it gets a positive consistency reward. If it disagrees, it gets penalized.

Example: If $R_{\tau_i} = 1.33$ and $S_{\tau_i, j} = +1$: consistency reward = $1.33 \times 1 = 1.33$ (correct, rewarded).
If $S_{\tau_i, j} = -1$: consistency reward = $1.33 \times (-1) = -1.33$ (incorrect, penalized).

### 3.3 Reward Precision Bound

As the number of PRM evaluations $m$ increases, the precision approaches 1:

$$A \geq 1 - \exp\left(-\frac{m(\mu - 1)^2}{4}\right)$$

where $\mu = p_+ + p_-$ is the sum of true positive and true negative rates.

Computationally: even a mediocre PRM (70% accuracy) becomes highly reliable with enough evaluations. With $m = 20$: precision > 95%.

In [ ]:
#@title 🎧 Code Walkthrough: Setup Build
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_04_setup_build.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

### 4.1 Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from typing import List, Tuple, Dict

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

In [ ]:
#@title 🎧 Code Walkthrough: Integrated Reward
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_05_integrated_reward.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 The Integrated Reward Computer

In [ ]:
class IntegratedRewardComputer:
    """
    Computes integrated rewards combining outcome and step-wise PRM signals.
    """

    def __init__(self, lambda_weight: float = 1.0, num_prm_votes: int = 3):
        self.lambda_weight = lambda_weight
        self.num_prm_votes = num_prm_votes

    def compute_integrated_reward(
        self,
        outcome: float,
        prm_scores: List[float]
    ) -> float:
        """
        Compute integrated reward for a single step.

        R_tau_i = O_tau + (lambda / m) * sum(S_tau_i_j)

        Args:
            outcome: Overall task outcome (-1 or +1)
            prm_scores: List of PRM evaluations for this step

        Returns:
            Integrated reward value
        """
        m = len(prm_scores)
        step_signal = self.lambda_weight / m * sum(prm_scores)
        return outcome + step_signal

    def compute_consistency_reward(
        self,
        integrated_reward: float,
        prm_score: float
    ) -> float:
        """
        Compute consistency reward for the PRM.

        R_S = R_tau_i * S_tau_i_j

        If PRM agrees with integrated reward → positive (PRM was right)
        If PRM disagrees → negative (PRM was wrong)

        Args:
            integrated_reward: The integrated reward for this step
            prm_score: This particular PRM evaluation

        Returns:
            Consistency reward for the PRM
        """
        return integrated_reward * prm_score

# Demonstrate
reward_computer = IntegratedRewardComputer(lambda_weight=1.0, num_prm_votes=3)

# Example 1: Successful task, PRM mostly agrees
outcome = +1.0
prm_scores = [+1, +1, -1]
R = reward_computer.compute_integrated_reward(outcome, prm_scores)
print(f"Example 1: Successful task, PRM scores = {prm_scores}")
print(f"  Integrated reward: R = {outcome} + (1/3)({sum(prm_scores)}) = {R:.2f}")
for j, s in enumerate(prm_scores):
    c = reward_computer.compute_consistency_reward(R, s)
    status = "✅ correct" if c > 0 else "❌ wrong"
    print(f"  PRM vote {j+1} (S={s:+d}): consistency = {R:.2f} × {s} = {c:.2f} ({status})")

print()

# Example 2: Failed task, PRM says step was good
outcome = -1.0
prm_scores = [+1, +1, +1]
R = reward_computer.compute_integrated_reward(outcome, prm_scores)
print(f"Example 2: Failed task, PRM scores = {prm_scores}")
print(f"  Integrated reward: R = {outcome} + (1/3)({sum(prm_scores)}) = {R:.2f}")
for j, s in enumerate(prm_scores):
    c = reward_computer.compute_consistency_reward(R, s)
    print(f"  PRM vote {j+1} (S={s:+d}): consistency = {R:.2f} × {s} = {c:.2f}")
print(f"  Note: R = {R:.2f}, PRM said +1. Product is {R:.2f} × 1 = {R:.2f}")
print(f"  This is slightly positive because the step might have been good")
print(f"  even though the overall task failed.")

In [ ]:
#@title 🎧 What to Look For: Reward Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_06_reward_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Integrated Rewards Across Different Scenarios

In [ ]:
scenarios = [
    {"outcome": +1, "prm_scores": [+1, +1, +1], "label": "Success + PRM agrees"},
    {"outcome": +1, "prm_scores": [+1, +1, -1], "label": "Success + PRM mostly agrees"},
    {"outcome": +1, "prm_scores": [-1, -1, +1], "label": "Success + PRM mostly disagrees"},
    {"outcome": -1, "prm_scores": [-1, -1, -1], "label": "Failure + PRM agrees"},
    {"outcome": -1, "prm_scores": [-1, +1, -1], "label": "Failure + PRM mostly agrees"},
    {"outcome": -1, "prm_scores": [+1, +1, +1], "label": "Failure + PRM disagrees"},
]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Integrated rewards
labels = [s["label"] for s in scenarios]
integrated = [reward_computer.compute_integrated_reward(s["outcome"], s["prm_scores"])
              for s in scenarios]
colors = ['#2ecc71' if r > 0 else '#e74c3c' for r in integrated]

y_pos = range(len(scenarios))
axes[0].barh(y_pos, integrated, color=colors, edgecolor='white', linewidth=2, alpha=0.8)
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(labels, fontsize=9)
axes[0].axvline(x=0, color='black', linewidth=0.5)
axes[0].set_xlabel('Integrated Reward')
axes[0].set_title('Integrated Rewards: Outcome + PRM', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Right: Consistency rewards for different PRM scores
consistency_data = {}
for r_val in np.linspace(-2, 2, 20):
    for s_val in [-1, +1]:
        key = f"S={s_val:+d}"
        if key not in consistency_data:
            consistency_data[key] = {"R": [], "C": []}
        consistency_data[key]["R"].append(r_val)
        consistency_data[key]["C"].append(r_val * s_val)

for key, data in consistency_data.items():
    color = '#2ecc71' if '+1' in key else '#e74c3c'
    axes[1].plot(data["R"], data["C"], linewidth=2.5, label=key, color=color)

axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].axvline(x=0, color='black', linewidth=0.5)
axes[1].set_xlabel('Integrated Reward (R)', fontsize=12)
axes[1].set_ylabel('Consistency Reward', fontsize=12)
axes[1].set_title('PRM Consistency Rewards', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Environment
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_07_environment.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The Simulated Environment with Difficulty Adaptation

In [ ]:
class AdaptiveEnvironment:
    """
    An environment that generates tasks and adapts difficulty
    based on the policy's success rate.
    """

    def __init__(self, vocab_size=50, seq_len=10,
                 difficulty: float = 0.3,
                 easy_threshold: float = 0.8,
                 hard_threshold: float = 0.2,
                 adapt_rate: float = 0.05):
        """
        Args:
            difficulty: Current difficulty level (0 = easiest, 1 = hardest)
            easy_threshold: If success > this, increase difficulty
            hard_threshold: If success < this, decrease difficulty
            adapt_rate: How much to change difficulty each adaptation step
        """
        self.vocab_size = vocab_size
        self.seq_len = seq_len
        self.difficulty = difficulty
        self.easy_threshold = easy_threshold
        self.hard_threshold = hard_threshold
        self.adapt_rate = adapt_rate
        self.success_history = deque(maxlen=50)
        self.difficulty_history = [difficulty]

    def generate_task(self, batch_size: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Generate a batch of tasks with current difficulty.

        Higher difficulty = more complex target patterns.

        Returns:
            (inputs, targets) tensors
        """
        inputs = torch.randint(0, self.vocab_size, (batch_size, self.seq_len)).to(device)

        # Difficulty controls target pattern complexity
        if self.difficulty < 0.3:
            # Easy: target is a simple repeating pattern
            base_pattern = torch.arange(self.seq_len) % 5
            targets = base_pattern.unsqueeze(0).expand(batch_size, -1).to(device)
        elif self.difficulty < 0.6:
            # Medium: target depends on input (first token modulates)
            modulator = inputs[:, 0:1] % 5
            base = torch.arange(self.seq_len).unsqueeze(0).to(device)
            targets = (base + modulator) % self.vocab_size
        else:
            # Hard: target is a complex function of multiple input tokens
            targets = (inputs * 3 + torch.roll(inputs, 1, dims=1) * 2) % self.vocab_size

        return inputs, targets

    def evaluate_outcome(self, predictions: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """
        Evaluate task outcomes.

        Returns:
            outcomes: (batch,) tensor of +1 (success) or -1 (failure)
        """
        match_rate = (predictions == targets).float().mean(dim=1)
        # Success threshold increases with difficulty
        threshold = 0.3 + 0.3 * self.difficulty
        outcomes = torch.where(match_rate > threshold,
                              torch.ones_like(match_rate),
                              -torch.ones_like(match_rate))
        return outcomes

    def adapt_difficulty(self, outcomes: torch.Tensor):
        """
        Adapt difficulty based on recent success rate.

        If success > 80%: tasks are too easy → increase difficulty
        If success < 20%: tasks are too hard → decrease difficulty
        """
        success_rate = (outcomes > 0).float().mean().item()
        self.success_history.append(success_rate)

        if len(self.success_history) >= 10:
            recent_success = np.mean(list(self.success_history)[-10:])

            if recent_success > self.easy_threshold:
                self.difficulty = min(1.0, self.difficulty + self.adapt_rate)
            elif recent_success < self.hard_threshold:
                self.difficulty = max(0.0, self.difficulty - self.adapt_rate)

        self.difficulty_history.append(self.difficulty)

    def get_stats(self):
        if len(self.success_history) > 0:
            return {
                "difficulty": self.difficulty,
                "recent_success": np.mean(list(self.success_history)[-10:]),
                "total_tasks": len(self.success_history),
            }
        return {"difficulty": self.difficulty, "recent_success": 0, "total_tasks": 0}

# Test the environment
env = AdaptiveEnvironment(difficulty=0.3)
inputs, targets = env.generate_task(batch_size=4)
print(f"Environment difficulty: {env.difficulty}")
print(f"Generated task shapes: inputs={inputs.shape}, targets={targets.shape}")
print(f"Sample input:  {inputs[0].tolist()}")
print(f"Sample target: {targets[0].tolist()}")

In [ ]:
#@title 🎧 Code Walkthrough: Prm Policy
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_08_prm_policy.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The Simulated PRM (Trainable)

In [ ]:
class TrainablePRM(nn.Module):
    """
    A trainable Process Reward Model.
    In production, this is a language model. Here we use a simple classifier.
    """

    def __init__(self, input_size=50, hidden_size=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 3),  # Output: [-1, 0, +1] logits
        )
        self.label_map = {0: -1, 1: 0, 2: +1}

    def forward(self, step_features: torch.Tensor) -> torch.Tensor:
        """
        Predict step quality.

        Args:
            step_features: (batch, input_size) — features describing a step

        Returns:
            logits: (batch, 3) — logits for [-1, 0, +1]
        """
        return self.net(step_features)

    def predict_score(self, step_features: torch.Tensor) -> torch.Tensor:
        """Get the predicted score (-1, 0, or +1)."""
        logits = self.forward(step_features)
        indices = logits.argmax(dim=-1)
        # Map 0→-1, 1→0, 2→+1
        scores = indices.float() - 1.0
        return scores

    def evaluate_with_voting(self, step_features: torch.Tensor, num_votes: int = 3):
        """
        Majority voting with dropout-based diversity.
        In production, this uses sampling. Here we use dropout.
        """
        self.train()  # Enable dropout for diversity
        votes = []
        for _ in range(num_votes):
            with torch.no_grad():
                score = self.predict_score(step_features)
                votes.append(score)

        self.eval()
        votes_stack = torch.stack(votes, dim=0)  # (num_votes, batch)

        # Majority vote per sample
        majority = []
        for b in range(step_features.shape[0]):
            sample_votes = votes_stack[:, b].tolist()
            from collections import Counter
            most_common = Counter(sample_votes).most_common(1)[0][0]
            majority.append(most_common)

        return torch.tensor(majority, device=step_features.device), votes_stack

# Test PRM
prm_model = TrainablePRM(input_size=50).to(device)
test_features = torch.randn(4, 50).to(device)
scores, votes = prm_model.evaluate_with_voting(test_features)
print(f"PRM scores: {scores.tolist()}")
print(f"PRM votes shape: {votes.shape}")

### 4.5 The Policy Model

In [ ]:
class PolicyModel(nn.Module):
    """Simple policy model for the closed-loop demo."""

    def __init__(self, vocab_size=50, hidden_size=64, seq_len=10):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.layers = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.head = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        h = self.layers(self.embed(x))
        return self.head(h)

    def predict(self, x):
        return self.forward(x).argmax(dim=-1)

    def get_log_probs(self, x, targets):
        logits = self.forward(x)
        log_probs = F.log_softmax(logits, dim=-1)
        selected = log_probs.gather(2, targets.unsqueeze(-1)).squeeze(-1)
        return selected

policy = PolicyModel().to(device)
ref_policy = PolicyModel().to(device)
ref_policy.load_state_dict(policy.state_dict())

print(f"✅ Policy model: {sum(p.numel() for p in policy.parameters()):,} params")
print(f"✅ PRM model:    {sum(p.numel() for p in prm_model.parameters()):,} params")

In [ ]:
#@title 🎧 Before You Start: Todo Section
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_09_todo_section.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. 🔧 Your Turn

### TODO 1: Implement the Consistency-Based PRM Update

The PRM improves by receiving consistency rewards. Implement the PRM training step:

In [ ]:
def train_prm_step(
    prm: TrainablePRM,
    prm_optimizer: torch.optim.Optimizer,
    step_features: torch.Tensor,    # (batch, feature_size)
    integrated_rewards: torch.Tensor, # (batch,) — integrated rewards for each step
    prm_scores: torch.Tensor,         # (batch,) — PRM's own predictions
) -> float:
    """
    Train the PRM using consistency feedback.

    The PRM should learn to agree with the integrated reward signal.
    When R * S > 0, the PRM was correct → reinforce this behavior.
    When R * S < 0, the PRM was wrong → correct this behavior.

    Args:
        prm: The trainable PRM model
        prm_optimizer: PRM optimizer
        step_features: Features describing each step
        integrated_rewards: Ground truth integrated rewards
        prm_scores: PRM's predictions for these steps

    Returns:
        Loss value

    Steps:
        1. Compute consistency rewards: R * S for each sample
        2. Convert integrated_rewards to classification targets:
           R > 0 → class 2 (+1), R == 0 → class 1 (0), R < 0 → class 0 (-1)
        3. Get PRM logits from forward pass
        4. Compute weighted cross-entropy loss (weight by |consistency_reward|)
        5. Backprop and step
    """
    # ============ TODO ============
    # Step 1: Create classification targets from integrated_rewards
    #   R > 0.5 → class 2, -0.5 < R < 0.5 → class 1, R < -0.5 → class 0
    # Step 2: Get PRM logits
    # Step 3: Compute cross-entropy loss
    # Step 4: Weight by |consistency_reward| for emphasis
    # Step 5: Backprop
    # ==============================

    loss = ???  # YOUR CODE HERE

    return loss

# ✅ Verification (run after implementing)
# prm_opt = torch.optim.Adam(prm_model.parameters(), lr=1e-3)
# test_features = torch.randn(8, 50).to(device)
# test_rewards = torch.tensor([1.33, -0.67, 1.0, -1.33, 0.33, 1.0, -1.0, 0.67]).to(device)
# test_scores = torch.tensor([1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0]).to(device)
# loss = train_prm_step(prm_model, prm_opt, test_features, test_rewards, test_scores)
# assert isinstance(loss, float), "❌ Should return float loss"
# print(f"✅ PRM training step works! Loss: {loss:.4f}")

### TODO 2: Implement the Difficulty Adaptation Logic

In [ ]:
def adapt_difficulty_step(
    env: AdaptiveEnvironment,
    recent_outcomes: torch.Tensor,
) -> Tuple[float, str]:
    """
    Decide whether to adapt environment difficulty.

    Args:
        env: The adaptive environment
        recent_outcomes: Recent task outcomes (+1/-1)

    Returns:
        (new_difficulty, action_taken)
        action_taken is one of: "increased", "decreased", "unchanged"

    Rules:
        - If recent success rate > env.easy_threshold → increase difficulty
        - If recent success rate < env.hard_threshold → decrease difficulty
        - Otherwise → no change
        - Difficulty must stay in [0, 1]
    """
    # ============ TODO ============
    # Step 1: Compute success rate from recent_outcomes
    # Step 2: Compare to thresholds
    # Step 3: Adjust env.difficulty accordingly
    # Step 4: Return new difficulty and action string
    # ==============================

    new_diff = ???  # YOUR CODE HERE
    action = ???    # YOUR CODE HERE

    return new_diff, action

# ✅ Verification
# test_env = AdaptiveEnvironment(difficulty=0.5)
# easy_outcomes = torch.ones(20)  # 100% success
# new_diff, action = adapt_difficulty_step(test_env, easy_outcomes)
# assert action == "increased", f"❌ Should increase, got {action}"
# assert new_diff > 0.5, f"❌ Difficulty should increase from 0.5"
# print(f"✅ Difficulty adaptation works! {0.5:.2f} → {new_diff:.2f} ({action})")

In [ ]:
#@title 🎧 Code Walkthrough: Closed Loop
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_10_closed_loop.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together — The Complete Closed Loop

In [ ]:
def run_closed_loop_training(
    num_steps: int = 200,
    batch_size: int = 16,
    policy_lr: float = 3e-4,
    prm_lr: float = 1e-3,
):
    """
    Run the complete RLAnything closed-loop training.

    Three components co-optimize:
    1. Policy learns to solve tasks better
    2. PRM learns to evaluate more accurately
    3. Environment adapts difficulty to keep things challenging
    """
    # Initialize all components
    env = AdaptiveEnvironment(difficulty=0.3)
    policy = PolicyModel().to(device)
    ref_policy = PolicyModel().to(device)
    ref_policy.load_state_dict(policy.state_dict())
    prm = TrainablePRM(input_size=50).to(device)

    policy_opt = torch.optim.Adam(policy.parameters(), lr=policy_lr)
    prm_opt = torch.optim.Adam(prm.parameters(), lr=prm_lr)

    reward_computer = IntegratedRewardComputer(lambda_weight=1.0, num_prm_votes=3)

    # Metrics tracking
    metrics = {
        "policy_success": [],
        "prm_accuracy": [],
        "difficulty": [],
        "integrated_rewards": [],
        "policy_loss": [],
    }

    print("🔄 Starting RLAnything closed-loop training...\n")

    for step in range(num_steps):
        # === Step 1: Environment generates tasks ===
        inputs, targets = env.generate_task(batch_size)

        # === Step 2: Policy generates predictions ===
        with torch.no_grad():
            predictions = policy.predict(inputs)
            outcomes = env.evaluate_outcome(predictions, targets)
            success_rate = (outcomes > 0).float().mean().item()

        # === Step 3: PRM evaluates each step ===
        # Create step features from (input, prediction, target)
        step_features = torch.zeros(batch_size, 50).to(device)
        step_features[:, :10] = inputs.float() / 50.0
        step_features[:, 10:20] = predictions.float() / 50.0
        step_features[:, 20:30] = targets.float() / 50.0

        with torch.no_grad():
            prm_scores, prm_votes = prm.evaluate_with_voting(step_features, num_votes=3)

        # === Step 4: Compute integrated rewards ===
        integrated_rewards = []
        for b in range(batch_size):
            prm_vote_list = prm_votes[:, b].tolist()
            R = reward_computer.compute_integrated_reward(
                outcomes[b].item(), prm_vote_list
            )
            integrated_rewards.append(R)
        integrated_rewards_t = torch.tensor(integrated_rewards, device=device)

        # === Step 5: Train policy with GRPO ===
        log_probs_new = policy.get_log_probs(inputs, targets)
        with torch.no_grad():
            log_probs_ref = ref_policy.get_log_probs(inputs, targets)

        # Compute advantages from integrated rewards
        mean_r = integrated_rewards_t.mean()
        std_r = integrated_rewards_t.std() + 1e-8
        advantages = (integrated_rewards_t - mean_r) / std_r

        ratio = torch.exp(log_probs_new - log_probs_ref.detach())
        adv_expanded = advantages.unsqueeze(1).expand_as(ratio)
        unclipped = ratio * adv_expanded
        clipped = torch.clamp(ratio, 0.8, 1.28) * adv_expanded
        policy_loss = -torch.min(unclipped, clipped).mean()

        policy_opt.zero_grad()
        policy_loss.backward()
        torch.nn.utils.clip_grad_norm_(policy.parameters(), 1.0)
        policy_opt.step()

        # === Step 6: Train PRM with consistency feedback ===
        # Target: PRM should agree with integrated reward direction
        prm_targets = torch.zeros(batch_size, dtype=torch.long, device=device)
        prm_targets[integrated_rewards_t > 0.5] = 2   # Class +1
        prm_targets[integrated_rewards_t < -0.5] = 0   # Class -1
        prm_targets[(integrated_rewards_t >= -0.5) & (integrated_rewards_t <= 0.5)] = 1  # Class 0

        prm_logits = prm(step_features)
        prm_loss = F.cross_entropy(prm_logits, prm_targets)

        prm_opt.zero_grad()
        prm_loss.backward()
        prm_opt.step()

        # Compute PRM accuracy
        with torch.no_grad():
            prm_preds = prm_logits.argmax(dim=-1)
            prm_accuracy = (prm_preds == prm_targets).float().mean().item()

        # === Step 7: Adapt environment difficulty ===
        env.adapt_difficulty(outcomes)

        # === Record metrics ===
        metrics["policy_success"].append(success_rate)
        metrics["prm_accuracy"].append(prm_accuracy)
        metrics["difficulty"].append(env.difficulty)
        metrics["integrated_rewards"].append(integrated_rewards_t.mean().item())
        metrics["policy_loss"].append(policy_loss.item())

        if (step + 1) % 50 == 0:
            print(f"  Step {step+1}/{num_steps}:")
            print(f"    Policy success: {success_rate:.1%}")
            print(f"    PRM accuracy:   {prm_accuracy:.1%}")
            print(f"    Difficulty:      {env.difficulty:.2f}")
            print(f"    Mean reward:     {integrated_rewards_t.mean():.2f}")
            print()

    print("✅ Closed-loop training complete!")
    return metrics, env

# Run the training
metrics, final_env = run_closed_loop_training(num_steps=200)

In [ ]:
#@title 🎧 What to Look For: Coevolution Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_11_coevolution_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. 📊 Visualizing the Co-Evolution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

window = 10

# 1. Policy success rate
success_smooth = np.convolve(metrics["policy_success"],
                              np.ones(window)/window, mode='valid')
axes[0, 0].plot(success_smooth, linewidth=2.5, color='#2ecc71')
axes[0, 0].set_ylabel('Success Rate', fontsize=12)
axes[0, 0].set_title('Policy Performance', fontsize=13, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylim(0, 1)

# 2. PRM accuracy
prm_smooth = np.convolve(metrics["prm_accuracy"],
                          np.ones(window)/window, mode='valid')
axes[0, 1].plot(prm_smooth, linewidth=2.5, color='#3498db')
axes[0, 1].set_ylabel('Accuracy', fontsize=12)
axes[0, 1].set_title('PRM Accuracy (co-evolving)', fontsize=13, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim(0, 1)

# 3. Environment difficulty
axes[1, 0].plot(metrics["difficulty"], linewidth=2.5, color='#e74c3c')
axes[1, 0].set_xlabel('Training Step', fontsize=12)
axes[1, 0].set_ylabel('Difficulty', fontsize=12)
axes[1, 0].set_title('Environment Difficulty (auto-adapting)', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim(0, 1)

# 4. All three together (normalized)
success_norm = np.array(metrics["policy_success"]) / max(metrics["policy_success"])
prm_norm = np.array(metrics["prm_accuracy"]) / max(max(metrics["prm_accuracy"]), 1e-8)
diff_norm = np.array(metrics["difficulty"]) / max(max(metrics["difficulty"]), 1e-8)

axes[1, 1].plot(np.convolve(success_norm, np.ones(window)/window, mode='valid'),
                linewidth=2, color='#2ecc71', label='Policy')
axes[1, 1].plot(np.convolve(prm_norm, np.ones(window)/window, mode='valid'),
                linewidth=2, color='#3498db', label='PRM')
axes[1, 1].plot(np.convolve(diff_norm, np.ones(window)/window, mode='valid'),
                linewidth=2, color='#e74c3c', label='Difficulty')
axes[1, 1].set_xlabel('Training Step', fontsize=12)
axes[1, 1].set_ylabel('Normalized Value', fontsize=12)
axes[1, 1].set_title('Co-Evolution: All Three Components', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('RLAnything Closed Loop — Co-Optimization Dynamics',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Precision Bound
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_12_precision_bound.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: The Reward Precision Bound

In [ ]:
def compute_precision_bound(m_values, mu):
    """Compute the reward precision lower bound."""
    return [1 - np.exp(-m * (mu - 1)**2 / 4) for m in m_values]

m_values = np.arange(1, 101)
fig, ax = plt.subplots(figsize=(10, 6))

for mu, color, label in [
    (1.1, '#e74c3c', 'μ=1.1 (barely better than random)'),
    (1.2, '#f39c12', 'μ=1.2 (mediocre PRM)'),
    (1.35, '#3498db', 'μ=1.35 (p+=0.7, p-=0.65)'),
    (1.5, '#2ecc71', 'μ=1.5 (good PRM)'),
    (1.8, '#9b59b6', 'μ=1.8 (excellent PRM)'),
]:
    precision = compute_precision_bound(m_values, mu)
    ax.plot(m_values, precision, linewidth=2, color=color, label=label)

ax.axhline(y=0.95, color='gray', linestyle='--', alpha=0.5, label='95% precision')
ax.set_xlabel('Number of PRM Evaluations (m)', fontsize=12)
ax.set_ylabel('Precision Lower Bound (A)', fontsize=12)
ax.set_title('Reward Precision Converges with More PRM Evaluations', fontsize=14, fontweight='bold')
ax.legend(fontsize=10, loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

# Specific numerical examples from the article
print("Precision bounds for μ=1.35 (p+=0.7, p-=0.65):")
for m in [5, 10, 20, 50, 100]:
    A = 1 - np.exp(-m * (1.35 - 1)**2 / 4)
    print(f"  m={m:3d}: A ≥ {A:.3f}")

In [ ]:
#@title 🎧 What to Look For: Final Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_13_final_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. 🎯 Final Output: The Complete RLAnything System

In [ ]:
fig = plt.figure(figsize=(16, 8))

# Create a visual summary of the closed-loop system
gs = fig.add_gridspec(2, 3, hspace=0.4, wspace=0.3)

# Top-left: Policy improvement
ax1 = fig.add_subplot(gs[0, 0])
success_smooth = np.convolve(metrics["policy_success"],
                              np.ones(10)/10, mode='valid')
ax1.fill_between(range(len(success_smooth)), success_smooth,
                 alpha=0.3, color='#2ecc71')
ax1.plot(success_smooth, linewidth=2, color='#2ecc71')
ax1.set_title('🎯 Policy', fontsize=12, fontweight='bold')
ax1.set_ylabel('Success Rate')
ax1.set_ylim(0, 1)

# Top-center: PRM improvement
ax2 = fig.add_subplot(gs[0, 1])
prm_smooth = np.convolve(metrics["prm_accuracy"],
                          np.ones(10)/10, mode='valid')
ax2.fill_between(range(len(prm_smooth)), prm_smooth,
                 alpha=0.3, color='#3498db')
ax2.plot(prm_smooth, linewidth=2, color='#3498db')
ax2.set_title('⚖️ Reward Model', fontsize=12, fontweight='bold')
ax2.set_ylabel('Accuracy')
ax2.set_ylim(0, 1)

# Top-right: Environment adaptation
ax3 = fig.add_subplot(gs[0, 2])
ax3.fill_between(range(len(metrics["difficulty"])), metrics["difficulty"],
                 alpha=0.3, color='#e74c3c')
ax3.plot(metrics["difficulty"], linewidth=2, color='#e74c3c')
ax3.set_title('🌍 Environment', fontsize=12, fontweight='bold')
ax3.set_ylabel('Difficulty')
ax3.set_ylim(0, 1)

# Bottom: Integrated reward evolution
ax4 = fig.add_subplot(gs[1, :])
reward_smooth = np.convolve(metrics["integrated_rewards"],
                             np.ones(10)/10, mode='valid')
ax4.plot(reward_smooth, linewidth=2.5, color='#9b59b6')
ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax4.set_xlabel('Training Step', fontsize=12)
ax4.set_ylabel('Mean Integrated Reward', fontsize=12)
ax4.set_title('Integrated Reward (Outcome + PRM)', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.suptitle('🎉 RLAnything: Complete Closed-Loop Co-Optimization',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Final statistics
final_success = np.mean(metrics["policy_success"][-20:])
final_prm_acc = np.mean(metrics["prm_accuracy"][-20:])
final_difficulty = metrics["difficulty"][-1]
initial_success = np.mean(metrics["policy_success"][:20])
initial_prm_acc = np.mean(metrics["prm_accuracy"][:20])

print("🎉 Congratulations! You've built the RLAnything closed loop from scratch!\n")
print("📊 Training Summary:")
print(f"   Policy success:  {initial_success:.1%} → {final_success:.1%} "
      f"({final_success - initial_success:+.1%})")
print(f"   PRM accuracy:    {initial_prm_acc:.1%} → {final_prm_acc:.1%} "
      f"({final_prm_acc - initial_prm_acc:+.1%})")
print(f"   Difficulty:      0.30 → {final_difficulty:.2f}")
print(f"\n   All three components improved together — this is co-optimization! 🚀")

In [ ]:
#@title 🎧 Wrap-Up: Reflection And Series Wrap
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_14_reflection_and_series_wrap.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### 🤔 Reflection Questions
1. What would happen if we made the PRM too powerful (100% accurate from the start)? Would co-optimization still help?
2. The difficulty adaptation uses fixed thresholds (80% and 20%). What would happen with tighter thresholds (e.g., 60% and 40%)? Would the system oscillate?
3. In the real OpenClaw-RL system, the user is the "environment" — they generate tasks by chatting. How does this differ from our simulated environment? What challenges does this introduce?

### 🏆 Optional Challenges
1. **Multi-step trajectories**: Extend the system to handle multi-step tasks where each step has its own integrated reward.
2. **PRM ensemble**: Instead of one PRM, train multiple PRMs and use their agreement as an additional signal.
3. **Catastrophic forgetting**: Add a regularization term that prevents the policy from forgetting old skills while learning new preferences.